In [ ]:
import os
from src import RASPRoutines
import polars as pl
import numpy as np
from src import IOFunctions
from src import AnalysisFunctions

RASP = RASPRoutines.RASP_Routines()
IO = IOFunctions.IO_Functions()
A_F = AnalysisFunctions.Analysis_Functions()

import warnings

warnings.filterwarnings("ignore")

In [ ]:
# MAKE SURE THESE POINT TO THE CORRECT FOLDERS
overall_folder = r"S:\ASAP_Imaging_Data\Main_Survey\20241105_oligomers_in_cells"
raw_data_folder = r"S:\ASAP_Imaging_Data\Main_Survey\Raw_data_ss"

In [ ]:
brainregion_folders = np.sort(os.listdir(overall_folder))

protein_strings = np.array(["C1", "C1", "C1", "C1", "C1", "C1", "C1", "C1"])
cell_strings = np.array(["C0", "C0", "C0", "C0", "C0", "C0", "C0", "C0"])

cell_stains = np.array(["microglia", "neurons", "astrocytes", "oligodendrocyte"])
patient_status = np.array(["PD", "HC"])
imtype = ".tif"
cell_size_threshold = np.array(
    [2000.0, 2000.0, 2000.0, 2000.0]
)  # first one for microglia, second for neurons, third for astrocytes, fourth for oligodendrocytes
percentiles = np.array(
    [
        0.0,
        5.0,
        10.0,
        20.0,
        25.0,
        30.0,
        35.0,
        40.0,
        45.0,
        50.0,
        55.0,
        60.0,
        65.0,
        70.0,
        75.0,
        80.0,
        85.0,
        90.0,
        95.0,
    ]
)

In [ ]:
for q, region in enumerate(brainregion_folders):
    protein_string = protein_strings[q]
    cell_string = cell_strings[q]
    for i, cell_stain in enumerate(cell_stains):
        for j, patient in enumerate(patient_status):
            """the command below will run the analyse_images analysis:

                folder (string): Folder containing images
                imtype (string): Type of images being analysed. Default '.tif'
                thres (float): fraction of bright pixels accepted. Default 0.05.
                large_thres (float): large object intensity threshold. Default 100.
                gisgma (float): gaussian blurring parameter. Default 1.4.
                rwave (float): Ricker wavelent sigma. Default 2.
                protein_string (np.1darray): strings for protein-stained data. Default C1.
                cell_string (np.1darray): strings for cell-containing data. Default C0.
                if_filter (boolean): Filter images for focus. Default True.
                im_start (integer): Images to start from. Default 0.
                cell_analysis (boolean): Parameter where script also analyses cell
                    images and computes colocalisation likelihood ratios. Default True.
                one_savefile (boolean): Parameter that, if true, doesn't save a file. Default True.
                    per image but amalgamates them into one file. Default True.
                disp (boolean): If true, prints when analysed an image stack. Default True.
                analyse_clr (boolean): If true, calculates the clr. If not, just coincidence. Default True.
                error_reduction (boolean): If true, reduces error on the oligomer image using Huang's code

            All have defaults except folder, which is necessary.
            """
        # then, get the percentile brightness from HC
        analysis_directory_HC = os.path.join(
            overall_folder, region, cell_stain, "HC" + "_analysis"
        )
        analysis_directory_PD = os.path.join(
            overall_folder, region, cell_stain, "PD" + "_analysis"
        )
        savename_HC = os.path.join(analysis_directory_HC, "spot_analysis.csv")
        savename_PD = os.path.join(analysis_directory_PD, "spot_analysis.csv")
        if os.path.isfile(savename_HC) and os.path.isfile(savename_PD):
            HC_data = pl.read_csv(savename_HC)
            PD_data = pl.read_csv(savename_PD)
            overall_region = pl.concat([HC_data, PD_data], rechunk=True)
            overall_region_filtered, q1, q2, IQR = A_F.reject_outliers(
                overall_region, k=4
            )
            thresholds = np.percentile(
                overall_region_filtered["sum_intensity_in_photons"].to_numpy(),
                percentiles,
            )
            if os.path.isfile(savename_HC):
                for patient in patient_status:
                    analysis_directory = os.path.join(
                        overall_folder, region, cell_stain, patient + "_analysis"
                    )
                    analysis_file = os.path.join(
                        analysis_directory, "spot_analysis.csv"
                    )
                    print("Analysing " + str(analysis_file))
                    analysis_data_raw, _, _, _ = A_F.reject_outliers(
                        pl.read_csv(analysis_file), q1=q1, q2=q2, IQR=IQR, k=4
                    )
                    for threshold in thresholds:
                        rdf = RASP.calculate_spot_rdf_with_threshold(
                            analysis_file,
                            analysis_data_raw,
                            threshold,
                        )